# Analysis of the risk vs returns of bitcoin

## Setup

In [ ]:
from adjustText import adjust_text
import matplotlib.colorbar as mpl_cbar
import matplotlib.colors as mpl_cols
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

In [ ]:
# Get all dfs
def get_df(csv_basename: str) -> pd.DataFrame:
    # Get df from CSV with date as index
    return pd.read_csv(f"../data/{csv_basename}.csv", index_col="date", parse_dates=True)

df_btc = get_df("BTC")
df_sp500 = get_df("SP500")
df_us10y = get_df("US10Y")
df_au = get_df("AU")
df_wti = get_df("WTI")
df_chf = get_df("CHF")

## Volatility vs returns (90-day rolling values)

In [ ]:
# Get daily logarithmic price change and 90-day moving average
df_btc["price_change_log"] = np.log(df_btc["price"] / df_btc["price"].shift(1))
df_btc["price_change_log_90d_ma"] = df_btc["price_change_log"].rolling(window=90).mean()

In [ ]:
# Get 90-day rolling volatility with the standard deviation method (for consistency)
df_btc["volatility_90d"] = df_btc["price_change_log"].rolling(window=90).std()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Normalize years to map colors
norm = mpl_cols.Normalize(vmin=df_btc.index.year.min(), vmax=df_btc.index.year.max())
cmap = plt.cm.viridis_r

sns.scatterplot(data=df_btc, x="volatility_90d", y="price_change_log_90d_ma", hue=df_btc.index.year, palette=cmap, alpha=0.7, linewidth=0.2, legend=False)
plt.axhline(y=0, color="red", linewidth=1, linestyle="--")

plt.title("Volatility vs returns of bitcoin")
plt.xlabel("Volatility (90-day window)")
plt.ylabel("Average daily returns (90-day window)")

# Create the colorbar
cax, _ = mpl_cbar.make_axes(ax, location="right", pad=0.01)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
cbar = fig.colorbar(sm, cax=cax)

save_chart_as_png("4_BTC_vlt_vs_returns")

In [ ]:
# Pearson correlation coefficient betweent volatility and returns
df_btc["volatility_90d"].corr(df_btc["price_change_log_90d_ma"]).round(2)

**Key takeaways:**
- ...

### Comparison of volatility vs returns with other assets

In [ ]:
# Get arithmetic returns for each df, instead of log returns since the oil data includes non-positive prices
for df in [df_btc, df_sp500, df_us10y, df_wti, df_au, df_chf]:
    df["returns"] = df.iloc[:, 0].pct_change()

In [ ]:
# Get df with average returns and std deviation of returns from all dfs since first data point of btc df
dfs = {
    "Bitcoin": df_btc,
    "S&P 500": df_sp500,
    "US 10-year yield": df_us10y,
    "Gold futures": df_au,
    "Crude oil futures": df_wti,
    "USD/CHF": df_chf,
}
results = []
for asset, df in dfs.items():
    results.append({"asset": asset,
                    "returns_avg": df.loc[df.index >= df_btc.index.min()]["returns"].mean(),
                    "returns_std": df.loc[df.index >= df_btc.index.min()]["returns"].std(),
                   })

df_comparison = pd.DataFrame(results)
df_comparison

In [ ]:
plt.figure(figsize=(10, 6))

sns.scatterplot(data=df_comparison, x="returns_std", y="returns_avg", color="lime")
plt.axhline(y=0, color="red", linewidth=0.5, linestyle="--")

plt.title("Comparison of the volatility vs returns of bitcoin with other assets since 2010")
plt.xlabel("Volatility of daily returns")
plt.ylabel("Average daily returns")

plt.xlim(0, 0.16)
plt.ylim(-0.003, 0.004)

# Add labels to each point
texts = []
for i, row in df_comparison.iterrows():
    texts.append(plt.text(row["returns_std"], row["returns_avg"], row["asset"], fontsize=10))

# Adjust text to avoid overlap
adjust_text(texts, arrowprops=dict(arrowstyle="-", color="white", alpha=2/3))

save_chart_as_png("4_BTC_vlt_vs_returns_comparison")

**Key takeaways:**
- ...